<img src="https://github.com/insaid2018/Term-1/blob/master/Images/INSAID_Full%20Logo.png?raw=true" width="240" height="360" />

# Recommendation Engine Part II - Singular Value Decomposition (SVD)

## Table of Contents

1. [Matrix Factorization](#section1)<br>
2. [Let's understand what is SVD](#section2)<br>
3. [What is the use of it?](#section3)<br>
4. [Loading Data](#section4)<br>
5. [Implementing Singular Vector Decoposition](#section5)<br>
6. [Setting up SVD](#section6)<br>
7. [Making Recommendations using SVD](#section7)<br>
8. [Model Evaluation](#section8)<br>

#### **Note: Kindly run this notebook on Google Colab to avoid Memory issues.**

<a id=section1></a>
## 1. Matrix Factorization

**Most important technique in recommendation system**<br><br>
- When a user gives feedback to a cerrtain movie they saw, this collection of feedback can be collected in the form of a matrix.
- Each row represents each users,
- Each column represents different movies.
- The matrix will be sparse since not everyone is going to watch every movies.

<img src = "https://raw.githubusercontent.com/insaid2018/Term-4/master/images/rec14.png">

The idea behind such models is that the preference of a user can be determined by a small number of hidden factors. We can call these factors as **Embeddings**.<br><br>

<a id=section2></a>
## 2. Let's understand what is SVD

Singular Value Decomposition(SVD) is a variability localization technique in which we represent data in form of matrix and then reduce the number of columns it has in order to maximize loss of dimensionality while minimizing loss of variability in the data being processed.<br>
Why wouldn’t the data be lost? The answer for that question is the essence of SVD.

Basically, SVD breaks a matrix into three other matrices called u, v, and d.

1- A is the real matrix with m*n elements.

2- U is an Orthogonal matrix with m*m elements

3- V is an Orthogonal matrix with n*n elements.

4- D is a diagonal matrix with m*n elements.

Orthogonal matrix is a matrix that does not get its properties changed if multiplied by other numbers.

<img src = "https://raw.githubusercontent.com/insaid2018/Term-4/master/images/svd.png">

<a id=section3></a>
## 3. What is the use of it?

When we decompose our matrix A into (U, D, V), a few left-most columns of all three matrices represent almost all the information we need to recover our actual data. For example 92% of the information in just 5% of total columns which is a pretty good deal given that you have reduced the size of your data set tremendously.

This means that SVD found some relation between all the columns of the matrix A and represented this same information with fewer columns.

The curse of dimensionality is no longer able to affect your performance.

**Matrix decompostion can be formulated as  an optimization problem with loss functions and constraints**

We can understand embeddings as low dimensional hidden factors for items and users.<br>
Let's say, we have 5 dimensional (D or n_factors = 5) embeddings for both items and users. Then for user-X and movie-A, we can say those 5 numbers might represent 5 different characterestics about the movies, like:
- How much movie-A is sci-fi intense?
- How recent is the movie?
- How much special effects ar in movie?
- How dialogue drive is the movie?


Like wise some numbers in user embedding matrix might represents,
- How much does user-X like sci-fi movies?
- How much does user-X like recent movies?

<img src= "https://raw.githubusercontent.com/insaid2018/Term-4/master/images/Shubham's%20crap.PNG">

- Source: [https://www.youtube.com/watch?v=ZspR5PZemcs](https://www.youtube.com/watch?v=ZspR5PZemcs)

<a id=section4></a>
## 4. Loading Data

In [0]:
# pip install scikit-surprise

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Reading ratings file
# Ignore the timestamp column
ratings = pd.read_csv('https://raw.githubusercontent.com/insaid2018/Term-4/master/Data/Assignment/ratings.csv', sep='\t', encoding='latin-1', usecols=['user_id', 'movie_id', 'rating'])


# Reading movies file
movies = pd.read_csv('https://raw.githubusercontent.com/insaid2018/Term-4/master/Data/Assignment/movies.csv', sep='\t', encoding='latin-1', usecols=['movie_id', 'title', 'genres'])

<a id=section5></a>
## 5. Implementing Singular Vector Decomposition

#### Using Ratings Data

In [0]:
n_users = ratings.user_id.unique().shape[0]
n_movies = ratings.movie_id.unique().shape[0]
print ('Number of users = ' + str(n_users) + ' | Number of movies = ' + str(n_movies))

Number of users = 6040 | Number of movies = 3706


- We want the format of my ratings matrix to be one row per user and one column per movie. 
- We'll pivot *ratings* to get that and call the new variable *Ratings* (with a capital *R).

In [0]:
Ratings = ratings.pivot(index = 'user_id', columns ='movie_id', values = 'rating').fillna(0)
Ratings.head()

We need to de-normalize the data (normalize by each users mean) and convert it from a dataframe to a numpy array.

In [0]:
R = Ratings.values
user_ratings_mean = np.mean(R, axis = 1)
Ratings_demeaned = R - user_ratings_mean.reshape(-1, 1)

- With the ratings matrix properly formatted and normalized, we can do some dimensionality reduction.

<a id=section6></a>
## 6. Setting Up SVD

Scipy and Numpy both have functions to do the singular value decomposition. We're going to use the Scipy function *svds* because it let's us choose how many latent factors we want to use to approximate the original ratings matrix (instead of having to truncate it after).

In [0]:
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(Ratings_demeaned, k = 50)

As we're going to leverage matrix multiplication to get predictions, We'll convert the $\Sigma$ (now are values) to the diagonal matrix form.

In [0]:
sigma = np.diag(sigma)

<a id=section7></a>
## 7. Making Recommendations using SVD

Now, we have everything we need to make movie ratings predictions for every user. We can do it all at once by following the math and matrix multiply $U$, $\Sigma$, and $V^{T}$ back to get the rank $k=50$ approximation of $A$.

But first, we need to add the user means back to get the actual star ratings prediction.

In [0]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)

With the predictions matrix for every user, we can build a function to recommend movies for any user. We return the list of movies the user has already rated, for the sake of comparison.

In [0]:
preds = pd.DataFrame(all_user_predicted_ratings, columns = Ratings.columns)
preds.head()

movie_id,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,3913,3914,3915,3916,3917,3918,3919,3920,3921,3922,3923,3924,3925,3926,3927,3928,3929,3930,3931,3932,3933,3934,3935,3936,3937,3938,3939,3940,3941,3942,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
0,4.288861,0.143055,-0.195080,-0.018843,0.012232,-0.176604,-0.074120,0.141358,-0.059553,-0.195950,0.512867,-0.089172,0.310181,-0.002005,-0.052401,-0.189827,0.238360,0.006466,-0.099315,-0.069682,-0.321492,0.111577,0.034795,0.320576,-0.118217,-0.012647,0.065573,-0.098318,0.064081,-0.005914,0.091936,0.180563,-0.009566,2.641693,-0.012495,0.765179,0.019784,0.002917,0.053079,0.014856,...,0.018810,-0.018782,0.022249,0.227852,-0.067653,-0.046039,-0.023574,-0.019405,-0.005116,-0.032921,-0.008259,-0.019157,0.007527,-0.008687,-0.025630,-0.013563,0.015240,-0.044665,-0.009568,-0.043549,-0.003131,-0.008221,-0.005948,0.031885,-0.003424,-0.001159,-0.002124,-0.002827,0.010393,-0.001068,0.027807,0.001640,0.026395,-0.022024,-0.085415,0.403529,0.105579,0.031912,0.050450,0.088910
1,0.744716,0.169659,0.335418,0.000758,0.022475,1.353050,0.051426,0.071258,0.161601,1.567246,0.772656,0.046179,-0.054562,0.042344,0.048390,0.347313,1.074905,-0.099782,0.008163,0.250869,2.186638,0.018789,-0.002199,0.218934,0.824475,0.139274,-0.007135,0.053071,-0.156952,0.044739,-0.002960,0.453298,-0.007484,0.920325,0.016566,1.335129,-0.015066,-0.045602,0.034649,0.122010,...,-0.042363,-0.137822,-0.112071,0.380783,-0.036273,-0.016174,0.002920,-0.148021,-0.017614,-0.033474,0.086133,0.008153,-0.126819,0.109208,0.001798,0.151866,0.014118,0.032897,0.005764,0.042259,0.022404,0.003260,0.010556,0.137181,-0.042184,0.006759,-0.005789,0.000340,0.002024,0.016013,-0.056502,-0.013733,-0.010580,0.062576,-0.016248,0.155790,-0.418737,-0.101102,-0.054098,-0.140188
2,1.818824,0.456136,0.090978,-0.043037,-0.025694,-0.158617,-0.131778,0.098977,0.030551,0.735470,-0.023476,0.034796,0.065942,0.008661,0.110348,-0.002952,-0.122061,0.063974,0.061033,0.081799,0.329471,0.149579,0.095352,-0.161493,0.022545,-0.009284,-0.002677,-0.142710,0.012345,-0.085331,0.076139,-0.355795,-0.008579,1.046871,-0.088946,0.383583,-0.018144,-0.038618,0.113984,0.006942,...,0.007233,-0.047221,0.066474,-0.179455,0.097428,0.034113,0.008098,-0.024784,-0.012749,-0.007394,-0.017220,0.004719,0.113348,-0.074943,-0.145795,0.128619,0.112567,0.045500,-0.018027,-0.058946,-0.002770,-0.035276,-0.008085,0.132182,-0.017005,0.014383,0.006598,-0.006217,-0.000342,0.000518,0.040481,-0.005301,0.012832,0.029349,0.020866,0.121532,0.076205,0.012345,0.015148,-0.109956
3,0.408057,-0.072960,0.039642,0.089363,0.041950,0.237753,-0.049426,0.009467,0.045469,-0.111370,-0.375831,0.068658,0.011199,0.069699,-0.037529,-0.238788,0.060607,-0.043418,0.053152,0.078237,0.357185,-0.096005,-0.028243,-0.067169,0.246164,-0.020379,0.034461,-0.022225,-0.012327,0.009182,0.014730,0.215893,-0.019687,-0.293933,-0.011511,0.145326,-0.029213,0.030029,-0.045409,-0.030684,...,-0.015077,-0.030208,0.028357,-0.072643,-0.135727,-0.053318,-0.012962,-0.054465,0.005870,-0.018048,-0.006836,-0.008222,-0.027214,-0.071677,-0.094072,-0.010745,-0.103191,-0.031297,-0.023920,-0.015053,-0.017914,-0.029561,-0.024299,-0.057678,-0.111450,-0.015473,-0.007123,-0.007416,-0.011508,-0.010038,0.008571,-0.005425,-0.008500,-0.003417,-0.083982,0.094512,0.057557,-0.026050,0.014841,-0.034224
4,1.574272,0.021239,-0.051300,0.246884,-0.032406,1.552281,-0.199630,-0.014920,-0.060498,0.450512,-0.251178,0.012337,-0.084051,0.258937,0.016570,0.980536,1.267869,0.275619,-0.008139,-0.038832,1.849627,0.107649,-0.168424,0.386541,1.790343,0.192379,-0.054356,0.267566,1.027817,0.374665,-0.010445,1.947980,0.017468,2.784035,0.274397,1.422393,0.040553,0.022926,1.345800,0.104507,...,0.075475,0.330767,0.150470,-0.261636,0.085163,-0.014229,-0.029247,0.124172,0.092875,0.061895,0.034757,0.054386,0.047055,0.048403,0.082926,0.129035,-0.174646,0.102727,0.024732,0.047280,0.017818,0.041451,0.041595,-0.007138,-0.080448,0.0

 Just to recall, below are the samples of movies and ratings dataset

In [0]:
movies.head(2)

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy


In [0]:
ratings.head(2)

,user_id,movie_id,rating
0,1,1193,5
1,1,661,3


Now, we write a function to return the movies with the highest predicted rating that the specified user hasn't already rated. Though we didn't use any explicit movie content features (such as genre or title), we'll merge in that information to get a more complete picture of the recommendations.

In [0]:
def recommend_movies(predictions, userID, movies, original_ratings, num_recommendations):
    
    # Get and sort the user's predictions
    user_row_number = userID - 1 # User ID starts at 1, not 0
    sorted_user_predictions = preds.iloc[user_row_number].sort_values(ascending=False) # User ID starts at 1
    
    # Get the user's data and merge in the movie information.
    user_data = original_ratings[original_ratings.user_id == (userID)]
    user_full = (user_data.merge(movies, how = 'left', left_on = 'movie_id', right_on = 'movie_id').
                     sort_values(['rating'], ascending=False)
                 )

    print('User {0} has already rated {1} movies.'.format(userID, user_full.shape[0]))
    print('Recommending highest {0} predicted ratings movies not already rated.'.format(num_recommendations))
    
    # Recommend the highest predicted rating movies that the user hasn't seen yet.
    recommendations = (movies[~movies['movie_id'].isin(user_full['movie_id'])].
         merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               left_on = 'movie_id',
               right_on = 'movie_id').
         rename(columns = {user_row_number: 'Predictions'}).
         sort_values('Predictions', ascending = False).
                       iloc[:num_recommendations, :-1]
                      )

    return user_full, recommendations

Let's try to recommend 20 movies for user with ID 1310.

In [0]:
already_rated, predictions = recommend_movies(preds, 1310, movies, ratings, 20)

User 1310 has already rated 24 movies.
Recommending highest 20 predicted ratings movies not already rated.


In [0]:
# Top 10 movies that User 1310 has rated 
already_rated.head(10)

,user_id,movie_id,rating,title,genres
5,1310,2248,5,Say Anything... (1989),Comedy|Drama|Romance
6,1310,2620,5,This Is My Father (1998),Drama|Romance
7,1310,3683,5,Blood Simple (1984),Drama|Film-Noir
15,1310,1704,5,Good Will Hunting (1997),Drama
1,1310,1293,5,Gandhi (1982),Drama
12,1310,3101,4,Fatal Attraction (1987),Thriller
11,1310,1343,4,Cape Fear (1991),Thriller
20,1310,2000,4,Lethal Weapon (1987),Action|Comedy|Crime|Drama
18,1310,3526,4,Parenthood (1989),Comedy|Drama
17,1310,3360,4,Hoosiers (1986),Drama


In [0]:
# Top 20 movies that User 1310 hopefully will enjoy
predictions

,movie_id,title,genres
1618,1674,Witness (1985),Drama|Romance|Thriller
1880,1961,Rain Man (1988),Drama
1187,1210,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Romance|Sci-Fi|War
1216,1242,Glory (1989),Action|Drama|War
1202,1225,Amadeus (1984),Drama
1273,1302,Field of Dreams (1989),Drama
1220,1246,Dead Poets Society (1989),Drama
1881,1962,Driving Miss Daisy (1989),Drama
1877,1957,Chariots of Fire (1981),Drama
1938,2020,Dangerous Liaisons (1988),Drama|Romance


- It's good to see that, although we didn't actually use the genre of the movie as a feature, the truncated matrix factorization features "picked up" on the underlying tastes and preferences of the user. 


- We've recommended some comedy, drama, and romance movies - all of which were genres of some of this user's top rated movies.

<a id=section8></a>
## 8. Model Evaluation

We will use the [Surprise](https://pypi.python.org/pypi/scikit-surprise) library that provided various ready-to-use powerful prediction algorithms including (SVD) to evaluate its **RMSE (Root Mean Squared Error)** on the MovieLens dataset. It is a Python scikit building and analyzing recommender systems.

In [0]:
# Import libraries from Surprise package
from surprise import Reader, Dataset, SVD#, evaluate

# Load Reader library
reader = Reader()

# Load ratings dataset with Dataset library
data = Dataset.load_from_df(ratings[['user_id', 'movie_id', 'rating']], reader)

# Split the dataset for 5-fold evaluation
# data.split(n_folds=5)

In [0]:
# from sklearn import model_selection
from surprise.model_selection import cross_validate

In [0]:
# Use the SVD algorithm.
svd = SVD()

# Compute the RMSE of the SVD algorithm.
cross_validate(svd, data, measures=['RMSE'], cv=5, verbose=True)

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8728  0.8754  0.8745  0.8742  0.8731  0.8740  0.0009  
Fit time          50.25   51.19   51.15   55.17   51.12   51.78   1.73    
Test time         2.82    2.77    2.35    2.74    2.71    2.68    0.17    


{'fit_time': (50.25301694869995,
  51.18598914146423,
  51.150800466537476,
  55.173394203186035,
  51.12203049659729),
 'test_rmse': array([0.87280177, 0.87541449, 0.87450958, 0.87418007, 0.87312575]),
 'test_time': (2.81990909576416,
  2.767259359359741,
  2.348268508911133,
  2.742522954940796,
  2.7103917598724365)}

- Root Mean Square Error of 0.8736 which is pretty good. 


- Now train on the dataset and arrive at predictions.

In [0]:
trainset = data.build_full_trainset()
svd.fit(trainset)

We'll pick again user with ID 1310 and check the ratings he has given.

In [0]:
ratings[ratings['user_id'] == 1310][:5]

,user_id,movie_id,rating
215928,1310,2988,3
215929,1310,1293,5
215930,1310,1295,2
215931,1310,1299,4
215932,1310,2243,4


Now let's use SVD to predict the rating that User with ID 1310 will give to a random movie (let's say with Movie ID 1994).

In [0]:
svd.predict(1310, 2988)

Prediction(uid=1310, iid=2988, r_ui=None, est=3.3680773794422, details={'was_impossible': False})

For movie with ID 1994, we get an estimated prediction of 3.349. The recommender system works purely on the basis of an assigned movie ID and tries to predict ratings based on how the other users have predicted the movie.

<a id=section9></a>
# 9. Conclusion

In this notebook, we attempted to build a movie recommendation sytem based on latent features from a low rank matrix factorization method called SVD. As it captures the underlying features driving the raw data, it can scale significantly better to massive datasets as well as make better recommendations based on user's tastes.

However, we still likely lose some meaningful signals by using a low-rank approximation. Specifically, there's an interpretability problem as a singular vector specifies a linear combination of all input columns or rows. There's also a lack of sparsity when the singular vectors are quite dense. Thus, SVD approach is limited to linear projections.